In [1]:
import numpy as np

from sudoku_stuff import *

# Resolviendo Sudokus con gradiente descendente
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

Dado que ya tenemos implementados los elementos necesarios para resolver Sudokus, vamos a resolver uno utilizando gradiente descendente discreto.

Comencemos con el siguiente problema:

<div>
<img src="./sudoku_7.png" width="300"/>
</div>

El diccionario de **celdas fijas** sería el siguiente:

In [2]:
fixed_squares = {
    'A1': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A7': 9,
    'B1': 1, 'B2': 8, 'B3': 5, 'B6': 9, 'B7': 7,
    'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D2': 2, 'D5': 1, 'D8': 4, 'D9': 9,
    'E2': 4, 'E3': 9, 'E5': 5, 
    'F3': 1, "F4": 9, "F5": 8, "F7": 6, "F8": 7,
    'G1': 4, 'G2': 9, 'G5': 3, 'G9': 7, 
    'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H9': 6,
    'I8': 8,
}

Y esta es la solución:

In [3]:
solution = {
    'A1': 3, 'A2': 7, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 1, 'A7': 9, 'A8': 2, 'A9': 8,
    'B1': 1, 'B2': 8, 'B3': 5, 'B4': 4, 'B5': 2, 'B6': 9, 'B7': 7, 'B8': 6, 'B9': 3,
    'C1': 9, 'C2': 6, 'C3': 2, 'C4': 3, 'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D1': 8, 'D2': 2, 'D3': 7, 'D4': 6, 'D5': 1, 'D6': 3, 'D7': 5, 'D8': 4, 'D9': 9,
    'E1': 6, 'E2': 4, 'E3': 9, 'E4': 2, 'E5': 5, 'E6': 7, 'E7': 8, 'E8': 3, 'E9': 1,
    'F1': 5, 'F2': 3, 'F3': 1, 'F4': 9, 'F5': 8, 'F6': 4, 'F7': 6, 'F8': 7, 'F9': 2,
    'G1': 4, 'G2': 9, 'G3': 6, 'G4': 8, 'G5': 3, 'G6': 2, 'G7': 1, 'G8': 5, 'G9': 7,
    'H1': 2, 'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H7': 3, 'H8': 9, 'H9': 6,
    'I1': 7, 'I2': 5, 'I3': 3, 'I4': 1, 'I5': 9, 'I6': 6, 'I7': 2, 'I8': 8, 'I9': 4,
}

In [4]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


## Implementando gradiente descendente

Este algoritmo se mueve continuamente en la dirección de mayor descenso del valor de la función de costo. La búsqueda termina cuando ninguno de los vecinos inmediatos tiene un costo menor. El algoritmo no examina más allá del vecindario directo.

Veamos la implementación, comentada paso a paso:

In [5]:
def gradient_descent_sudoku(initial_state: dict, fixed_squares: dict, max_iterations: int = 1000) -> tuple:
    """
    Realiza la optimización del Sudoku utilizando el método de descenso de gradiente.

    Args:
        initial_state (dict): El estado inicial del Sudoku.
        fixed_squares (dict): Diccionario que contiene las casillas fijas del Sudoku.
        max_iterations (int, opcional): El número máximo de iteraciones permitidas.
                                        Por defecto es 1000.

    Returns:
        dict: El mejor estado encontrado después de la optimización.
        float: El costo del mejor estado encontrado.
    """
    best_state = initial_state
    cost_state = cost_function(best_state)

    # Iteramos hasta alcanzar el máximo de iteraciones
    for iteration in range(max_iterations):

        # Calculamos la función de costo para el estado actual
        cost_state = cost_function(best_state)

        # Si el costo es cero, significa que estamos en un mínimo (la solución)
        if cost_state == 0:
            break

        # Obtenemos los vecinos inmediatos
        neib_states = return_neib_states(best_state, fixed_squares)

        # Calculamos el cambio de costo entre el estado actual y cada vecino
        neib_energy_list = [cost_function(neib_state) - cost_state for neib_state in neib_states]

        # Encontramos el índice del vecino con la mayor reducción de costo
        index_min_energy = np.argmin(neib_energy_list)

        # Si no hay ningún vecino que reduzca el costo, alcanzamos un mínimo local
        if neib_energy_list[index_min_energy] >= 0:
            return best_state, cost_state

        # Si hay mejora, nos movemos al mejor vecino
        best_state = neib_states[index_min_energy]

    # Retornamos el mejor estado alcanzado tras las iteraciones
    return best_state, cost_state

Ahora veamos si podemos encontrar la solución en una única ejecución. Para ello, vamos a utilizar una función llamada `execute_search()`, a la cual le pasamos la implementación del algoritmo. Esta función inicializa el Sudoku en un estado aleatorio, aplica la búsqueda y verifica si se ha alcanzado una solución o no.


In [6]:
from processing import execute_search

In [7]:
# El primer número es un identificador que, por ahora, no nos importa qué valor asuma.
solution_bool, last_state, initial_state, _ = execute_search(0, gradient_descent_sudoku, fixed_squares)

In [8]:
print(f"Valor de costo inicial: {cost_function(initial_state)}; valor final: {cost_function(last_state)}")
print("Primer estado:")
print_state(initial_state)

print("Último estado encontrado:")
print_state(last_state)

Valor de costo inicial: 6.8; valor final: 1.5000000000000002
Primer estado:
*---------+---------+---------*
| 3  5  4 | 5  6  4 | 9  8  8 |
| 1  8  5 | 6  3  9 | 7  5  8 |
| 2  2  5 | 2  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  4 | 6  1  7 | 5  4  9 |
| 2  4  9 | 7  5  4 | 8  3  2 |
| 9  1  1 | 9  8  6 | 6  7  3 |
*---------+---------+---------*
| 4  9  3 | 8  3  7 | 2  3  7 |
| 4  1  8 | 7  4  5 | 1  7  6 |
| 8  3  9 | 8  1  3 | 4  8  8 |
*---------+---------+---------*
Último estado encontrado:
*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 3  2  9 | 7  6  4 |
| 2  6  9 | 4  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  6 | 3  1  7 | 5  4  9 |
| 7  4  9 | 1  5  6 | 8  3  2 |
| 9  5  1 | 9  8  4 | 6  7  3 |
*---------+---------+---------*
| 4  9  3 | 8  3  1 | 2  5  7 |
| 5  1  8 | 7  4  5 | 3  9  6 |
| 6  7  2 | 2  9  3 | 4  8  1 |
*---------+---------+---------*


In [9]:
print("¿El estado encontrado es una solución?")
if solution_bool:
    print("El estado que encontramos verifica correctamente que es la solución.")
else:
    print("El estado que encontramos no es una solución.")

¿El estado encontrado es una solución?
El estado que encontramos no es una solución.


Vemos que el algoritmo no está encontrando la **solución**. Ahora podemos probar repitiendo 500 ejecuciones de la búsqueda, comenzando desde distintos puntos iniciales, para ver si eventualmente llegamos a la solución. Para acelerar el proceso, aprovecharemos que disponemos de **CPUs multinúcleo**.

Para ello, utilizaremos la función `parallel_sudoku_search()`, a la cual le pasaremos la función de gradiente descendente y la cantidad de iteraciones que queremos realizar.

> 🧠 Nota: Deben instalar `tqdm` para visualizar el progreso de las ejecuciones. Si están usando conda, pueden instalarlo ejecutando el siguiente comando:
> `conda install -y tqdm`

In [10]:
from processing import parallel_sudoku_search

# Debemos llamar a la función de búsqueda desde un archivo .py, ya que los threads no pueden recibirla directamente desde la notebook.
from search_methods import gradient_descent_sudoku

In [11]:
results = parallel_sudoku_search(gradient_descent_sudoku, fixed_squares, max_iterations=500)

  0%|          | 0/500 [00:00<?, ?it/s]

Veamos si alguno de los procesos encontró la solución:

In [12]:
show_solution = True
for res in results:
    # El primer valor indica si se encontró la solución o no
    is_solution = res[0]
    # Este es el último estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde el cual se inició
    initial_state = res[2]
    # Identificador de la iteración en la que se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 11 se encontró la solución.
En la iteración 317 se encontró la solución.


---
## Gradiente descendente estocástico

Vemos que, con el gradiente descendente, resulta difícil encontrar la solución. Una variante de este algoritmo es el gradiente descendente estocástico, en el cual, en lugar de movernos siempre en la dirección de máxima pendiente (o derivada), nos movemos en una dirección aleatoria que igualmente mantenga el descenso en el valor de la función de costo.

Veamos la implementación, prestando atención a los comentarios en detalle:

In [13]:
def gradient_descent_random_sudoku(initial_state: dict, fixed_squares: dict, max_iterations: int = 1000) -> tuple:
    """
    Realiza la optimización del Sudoku utilizando el método de descenso de gradiente estocástico.

    Args:
        initial_state (dict): El estado inicial del Sudoku.
        fixed_squares (dict): Diccionario que contiene las casillas fijas del Sudoku.
        max_iterations (int, opcional): El número máximo de iteraciones permitidas. Por defecto es 1000.

    Returns:
        dict: El mejor estado encontrado después de la optimización.
        float: El costo del mejor estado encontrado.
    """
    best_state = initial_state
    cost_state = cost_function(best_state)

    # Iteramos hasta alcanzar el número máximo de iteraciones
    for iteration in range(max_iterations):

        # Calculamos la función de costo para el estado actual
        cost_state = cost_function(best_state)

        # Si el costo es cero, significa que hemos alcanzado un mínimo. Esto tiene sentido en el contexto del Sudoku
        # y la función de costo que implementamos.
        if cost_state == 0:
            break

        # Obtenemos los vecinos más cercanos
        neib_states = return_neib_states(best_state, fixed_squares)

        # Calculamos el delta de costo entre el estado actual y cada uno de sus vecinos
        neib_energy_list = [cost_function(neib_state) - cost_state for neib_state in neib_states]

        # Obtenemos los índices de los vecinos con un costo menor (descenso)
        index_min_energy = [i for i, x in enumerate(neib_energy_list) if x < 0]

        # Si no hay vecinos que mejoren el estado, retornamos el mejor estado encontrado
        if not index_min_energy:
            return best_state, cost_state

        # Si los hay, elegimos uno al azar entre ellos
        index_sel = random.choice(index_min_energy)
        best_state = neib_states[index_sel]

    # Si se completan todas las iteraciones, retornamos el mejor resultado alcanzado
    return best_state, cost_state

Ahora probamos ejecutar el algoritmo:

In [14]:
solution_bool, last_state, initial_state, _ = execute_search(0, gradient_descent_random_sudoku, fixed_squares)

In [15]:
print(f"Valor de costo inicial: {cost_function(initial_state)}; valor final: {cost_function(last_state)}")
print("Primer estado:")
print_state(initial_state)

print("Último estado encontrado:")
print_state(last_state)

Valor de costo inicial: 7.3; valor final: 0.4
Primer estado:
*---------+---------+---------*
| 3  6  4 | 5  6  1 | 9  8  8 |
| 1  8  5 | 1  7  9 | 7  9  8 |
| 5  5  5 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 3  2  8 | 8  1  1 | 8  4  9 |
| 9  4  9 | 4  5  5 | 8  9  7 |
| 2  9  1 | 9  8  2 | 6  7  8 |
*---------+---------+---------*
| 4  9  3 | 3  3  2 | 1  6  7 |
| 2  1  8 | 7  4  5 | 9  8  6 |
| 1  5  4 | 6  5  7 | 5  8  9 |
*---------+---------+---------*
Último estado encontrado:
*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 6  2  7 | 2  1  3 | 8  4  9 |
| 8  4  9 | 6  5  7 | 3  5  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  9  7 |
| 2  1  8 | 7  4  5 | 5  3  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


In [16]:
print("¿El estado encontrado es una solución?")
if solution_bool:
    print("El estado que encontramos verifica correctamente que es la solución.")
else:
    print("El estado que encontramos no es una solución.")

¿El estado encontrado es una solución?
El estado que encontramos no es una solución.


Veamos ahora qué ocurre al ejecutar varias veces:

In [17]:
from search_methods import gradient_descent_random_sudoku

results = parallel_sudoku_search(gradient_descent_random_sudoku, fixed_squares, max_iterations=500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [18]:
show_solution = True
for res in results:
    # Acá se devuelve un booleano que indica si se encontró la solución
    is_solution = res[0]
    # Este es el último estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partió la búsqueda
    initial_state = res[2]
    # Este es el identificador de la iteración en la que se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 20 se encontró la solución.
En la iteración 303 se encontró la solución.


### Movernos en mesetas

La última variante que vamos a aplicar en este caso consiste en permitir que, en el gradiente descendente estocástico, se puedan seleccionar vecinos cuya diferencia de costo con respecto al estado actual sea cero. Esto nos permite desplazarnos dentro de mesetas, con la esperanza de alcanzar un precipicio (una bajada abrupta del costo) y continuar reduciendo el valor de la función de costo.

Para ello, modificamos la función `gradient_descent_random_sudoku()` —definida en `search_methods`— agregándole un nuevo parámetro llamado `move_in_zero`, el cual cambia una sola línea clave de la función:


```python
index_min_energy = [i for i, x in enumerate(neib_energy_list) if x <= 0]
```

La idea es conservar todos los vecinos que mantengan o reduzcan el costo, y luego seleccionar uno de ellos de manera aleatoria.

In [19]:
from search_methods import gradient_descent_random_sudoku
from functools import partial

new_grad_desc = partial(gradient_descent_random_sudoku, move_in_zero=True)

results = parallel_sudoku_search(new_grad_desc, fixed_squares, max_iterations=500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [20]:
show_solution = True
for res in results:
    # Acá se devuelve un booleano que indica si se encontró la solución
    is_solution = res[0]
    # Este es el último estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partió la búsqueda
    initial_state = res[2]
    # Este es el identificador de la iteración en la que se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 0 se encontró la solución.
En la iteración 1 se encontró la solución.
En la iteración 2 se encontró la solución.
En la iteración 3 se encontró la solución.
En la iteración 5 se encontró la solución.
En la iteración 6 se encontró la solución.
En la iteración 7 se encontró la solución.
En la iteración 8 se encontró la solución.
En la iteración 9 se encontró la solución.
En la iteración 10 se encontró la solución.
En la iteración 11 se encontró la solución.
En la iteración 12 se encontró la solución.
En la iteración 13 se encontró la solución.
En la iteración 16 se

Vemos que el algoritmo fue más eficiente al incorporar este cambio. Moverse en la meseta resultó ser la clave del éxito.

¿Por qué? Porque en el caso de resolver Sudokus, permitir que el algoritmo de gradiente descendente se desplace por zonas donde el costo se mantiene constante le da la posibilidad de escapar de mesetas y explorar regiones del espacio de búsqueda que eventualmente conducen a la solución.
